### 1. combine `mon_features.pkl` & `unmon_features.pkl` into `features_df`

In [ ]:
import pandas as pd
import numpy as np
import pickle

MON_FILE_PATH = '/content/mon_features.pkl'
UNMON_FILE_PATH = '/content/unmon_features.pkl'

LABEL_COLUMN = ['website_label', 'monitored_label']

FEATURES_VER2 = [
    'total_transmission_time', 'std_inter_packet_time',
    'avg_outgoing_burst_size', 'avg_incoming_burst_size',
    'num_outgoing_packets', 'incoming_packet_ratio',
    'outgoing_packet_ratio', 'cumul_packets_10pct',
    'cumul_packets_30pct', 'outgoing_order_skew',
    'incoming_order_skew', 'cumul_max', 'bigram_OO',
    'num_incoming_first_30', 'outgoing_first_30',
    'avg_incoming_order_first_30', 'avg_outgoing_order_first_30'
]

mon_features_df = pd.read_pickle(MON_FILE_PATH)
unmon_features_df = pd.read_pickle(UNMON_FILE_PATH)

features_df = pd.concat([mon_features_df, unmon_features_df], ignore_index=True)

X = features_df[FEATURES_VER2]
y = features_df[LABEL_COLUMN[1]]

y = y.replace({-1: 0, 1: 1})

print(X)
print(y)

       total_transmission_time  std_inter_packet_time  \
0                        10.14               0.041168   
1                        10.16               0.163930   
2                        11.11               0.066661   
3                        13.36               0.047809   
4                        10.64               0.038760   
...                        ...                    ...   
28995                    32.09               0.163669   
28996                    38.62               0.114350   
28997                    34.93               1.331199   
28998                    11.84               0.083521   
28999                     9.62               0.026874   

       avg_outgoing_burst_size  avg_incoming_burst_size  num_outgoing_packets  \
0                     1.551282                16.666667                 121.0   
1                     1.702128                 9.319149                  80.0   
2                     1.552632                16.315789                 

### 2. XGBoost

#### (1) install

In [ ]:
pip install xgboost

In [ ]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score

#### (2) Split dataset

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

#### (3) XGBoost 모델 선언

In [ ]:
model = xgb.XGBClassifier(
    n_estimators=400,
    learning_rate=0.03,
    max_depth=5,
    subsample=0.8,
    colsample_bytree=0.8,
    objective='binary:logistic',   # binary classification
    eval_metric='logloss',
    random_state=42,
    tree_method='hist',            # 속도 빠르고 메모리 효율적
)

#### (4) 모델 학습

In [ ]:
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.03, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=400, n_jobs=None,
              num_parallel_tree=None, ...)

#### (5) 예측

In [ ]:
y_pred = model.predict(X_test)
y_proba = model.predict_proba(X_test)[:, 1]  # positive 확률값

#### (6) 평가

In [ ]:
print("Accuracy:", accuracy_score(y_test, y_pred))
print("ROC-AUC:", roc_auc_score(y_test, y_proba))
print("\n📄 Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.9741379310344828
ROC-AUC: 0.9969873684210526

📄 Classification Report:
               precision    recall  f1-score   support

           0       0.97      0.96      0.96      2000
           1       0.98      0.98      0.98      3800

    accuracy                           0.97      5800
   macro avg       0.97      0.97      0.97      5800
weighted avg       0.97      0.97      0.97      5800

